In [ ]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '/Users/richamishra/Documents/CompPhoto/assgn6/src')

from cp_hw6 import *
from glob import glob



In [ ]:
#Get the extinsic and intrinsic parameters

#Checkerboard images
fnames = sorted(glob("/Users/richamishra/Documents/CompPhoto/assgn6/data/data_calib/*.JPG"))
mtx, dist = computeIntrinsic(fnames, (6,8), (8,8))
np.savez("/Users/richamishra/Documents/CompPhoto/assgn6/data/data_calib/intrinsic_calib.npz", mtx=mtx, dist=dist)

with np.load("/Users/richamishra/Documents/CompPhoto/assgn6/data/data_calib/intrinsic_calib.npz") as X:
    mtx, dist = [X[i] for i in ('mtx', 'dist')]
    
#Get the extrinsic parameters
#tvec_v, r_mat_v = computeExtrinsic("/Users/richamishra/Documents/CompPhoto/assgn6/DSC_0062.JPG", mtx, dist, 300, 166)
#tvec_h, r_mat_h = computeExtrinsic("/Users/richamishra/Documents/CompPhoto/assgn6/DSC_0062.JPG", mtx, dist, 300, 166)
#ext_out = {"tvec_h":tvec_h, "rmat_h":rmat_h, "tvec_v":tvec_v, "rmat_v":rmat_v}
#np.savez(os.path.join(baseDir, objName, seqName, "extrinsic_calib.npz"), **ext_out)

In [ ]:
# Extract frames

SAVING_FRAMES_PER_SECOND = 60

def get_saving_frames_durations(cap, saving_fps):
    """A function that returns the list of durations where to save the frames"""
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

def extract_frames(video_file, foldername, save_fps=SAVING_FRAMES_PER_SECOND):
    print("save to :",foldername)
    # make a folder by the name of the video file
    if not os.path.isdir(foldername):
        os.mkdir(foldername)
        
    # read the video file   
    assert os.path.isfile(video_file)
    cap = cv2.VideoCapture(video_file)
    print(f"#frames:{cap.get(cv2.CAP_PROP_FRAME_COUNT)}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, "
          f"height:{cap.get(cv2.CAP_PROP_FRAME_HEIGHT)}, width:{cap.get(cv2.CAP_PROP_FRAME_WIDTH)}")
    
    # get the FPS of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(fps, save_fps)
    # get the list of duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    print(f"Total saving frames: {len(saving_frames_durations)}")
    
    # start the loop
    frame_count = 0
    save_count = 0
    with tqdm(total=len(saving_frames_durations)) as pbar:
        while len(saving_frames_durations): # if the list is empty, all duration frames were saved
            is_read, frame = cap.read()
            if not is_read:
                break # break out of the loop if there are no frames to read

            # get the duration by dividing the frame count by the FPS
            frame_duration = frame_count / fps
            frame_count += 1 # increment the frame count
            closest_duration = saving_frames_durations[0] # get the earliest duration to save
            
            # if closest duration is less than/equals the frame duration, then save the frame
            if frame_duration >= closest_duration:
#                 frame = cv2.cvtColor(center_crop_resize(frame), cv2.COLOR_RGB2BGR)
#                 frame = (frame*255).astype(np.uint8)
#                 plt.imshow(frame)
#                 plt.show()
                
                cv2.imwrite(os.path.join(foldername, "{:04}.jpg").format(save_count), frame) 
                saving_frames_durations.pop(0)
                save_count += 1
                pbar.update()
                
video_file = "data/data3/DSC_0066.MOV"
foldername = "data/data3/"

extract_frames(video_file, foldername)

In [ ]:
import numpy as np
import cv2
import skimage
import matplotlib.pyplot as plt
import scipy
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from tqdm import tqdm
import os

In [ ]:
frog_path = r"./data/data4/"
#calib_path = r"./data/data_calib/"
#calib_lr_path = r"./data/calib-lr/"

def read_images(path):
    images = []
    paths = sorted(glob(path + "*.jpg"))
    for image_path in paths:
        images.append(np.asarray(skimage.io.imread(image_path)))
    return np.array(images)

def transform2gray(images):
    gray_images = []
    n = images.shape[0]
    for i in range(n):
        gray_images.append(skimage.color.rgb2gray(images[i, :, :, :]))
        
    return np.array(gray_images)

def get_diff(images):
    
    max_I = np.max(images, axis = 0)
    min_I = np.min(images, axis = 0)
    I_s = (max_I + min_I) / 2
    diff = images - I_s
        
    return np.array(diff)


def points2planes(diff, col_d, row_d):
    planes = []
    for i in tqdm(range(images.shape[0])):
        image = diff[i, :, :]
        H, W = image.shape
        points_h = np.zeros((image.shape[0], 3))
        for r in range(H):
            delta = np.diff(np.sign(image[r, :]))
            zero_crossings = np.where(delta > 0)[0]
            zero_col = zero_crossings[-1]
            points_h[r, :] = np.array([zero_col + col_d, r + row_d, 1])

        U, S, Vh = np.linalg.svd(points_h)
        planes.append(Vh[-1, :])
        
    planes = np.array(planes)
    return planes

def get_pixel_shadow_time_estimation(images):
    
    t, h, w = images.shape
    diff_im = np.diff(np.sign(images), axis=0, prepend=np.ones((1,h,w)))
    t_cross = np.argmax(diff_im, axis=0)
    
    return t_cross

def project_rays(p_points, mtx, dist):

    p_points = p_points.reshape(-1, 2).astype(np.float32)
    p_rays = pixel2ray(p_points, mtx, dist)
    
    return p_rays

def camera2plane(P, r, t):

    #P_plane = R.T (P_camera - T)
    # P_camera (nx1x3)
    P = P.reshape(-1, 3).T # (3, n)
    t = t.reshape(3, 1)
    return (r.T @ (P - t)).reshape(-1, 3)

def get_intersection(rays_n, ray_origins):

    o1, o2, o3 = ray_origins[0], ray_origins[1], ray_origins[2]
    d = - o3 / rays_n[:, 2]
    x = o1 + (d*rays_n[:, 0])
    y = o2 + (d*rays_n[:, 1])
    z = np.zeros_like(x)
    points = np.stack((x, y, z), axis=1)
    return points

def get3Dpoints(plane_n, plane_o, ray_n):

    d_n = plane_n[:, 0]*(plane_o[:, 0] - ray_n[:, 0]) + plane_n[:, 1]*(plane_o[:, 1] - ray_n[:, 1]) + \
        plane_n[:, 2]*(plane_o[:, 2] - ray_n[:, 2])
    d_d = plane_n[:, 0] * ray_n[:, 0] + plane_n[:, 1] * ray_n[:, 1] + plane_n[:, 2] * ray_n[:, 2]
    d = d_n / d_d
    
    
    x = (d * ray_n[:, 0])
    y = (d * ray_n[:, 1])
    z = (d * ray_n[:, 2])

    pts_3D = np.vstack((x, y, z)).T # (n, 3)
    return pts_3D

def calibration_shadow_planes(p1p2, p3p4):

    p1s = p1p2[0::2]
    p2s = p1p2[1::2]
    p3s = p3p4[0::2]
    p4s = p3p4[1::2]
    
    lines1 = p2s - p1s 
    lines2 = p4s - p3s
        
    n_params = np.cross(lines1, lines2, axisa=-1, axisb=-1)
    n_norm = np.sqrt(np.sum(n_params**2, axis = -1))
    normal = n_params/n_norm.reshape(-1, 1)
    origin = p1s
    
    return origin, normal

def final_reconstruction(im, pixel_times, S_points, S_normals, mtx, dist, max_I, min_I):

    # crop region
    x1, x2, y1, y2 = 320, 620, 310, 800
    points_x, points_y = np.meshgrid(np.arange(x1, x2), np.arange(y1, y2))
    
    pixels = im[points_x, points_y, :]
    pixels_max = max_I[points_x, points_y]
    pixels_min = min_I[points_x, points_y]
    
    pixels = pixels.reshape((-1, 3))
    points = np.concatenate([points_x, points_y], axis=-1).reshape((-1,2))
    ts = pixel_times[points_x, points_y]
    p, n = S_points[ts].reshape((-1, 3)), S_normals[ts].reshape((-1, 3))
      
    rays = project_rays(points, mtx, dist).reshape((-1, 3))
    
    pts_3D = get3Dpoints(n, p, rays)

    id1, _ = np.where((pixels_max - pixels_min < 0.5))
    fig = plt.figure("Reconstruction Results")
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    pts_3D_new = pts_3D[id1, :]
    pixels_new = pixels[id1, :]/255
    np.save(r"./reconstructed_points_" + str(0.5) + r".npy", pts_3D_new)
    np.save(r"./reconstructed_colors_" + str(0.5) + r".npy", pixels_new)
    
    ax.scatter(pts_3D_new[:, 0], pts_3D_new[:, 1], pts_3D_new[:, 2], \
        c=pixels_new, s=10, marker="s")
    plt.show()

In [ ]:
frog_images = read_images(frog_path)
frog_images = transform2gray(frog_images)
I_delta = get_diff(frog_images)

